In [2]:
import math,torch,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

from n_framework import *

from fastprogress import progress_bar,master_bar

In [3]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn,tensor
from datasets import load_dataset,load_dataset_builder
from n_framework import *
import logging
from fastcore.test import test_close

In [4]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [5]:
logging.disable(logging.WARNING)

## Learner

In [6]:
x,y = 'image','label'
name = 'fashion_mnist'
dsd = load_dataset(name)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
@inplace
def transformi(b): b[x]=[torch.flatten(TF.to_tensor(o))for o in b[x]]

In [8]:
bs=1024
tds = dsd.with_transform(transformi)

In [61]:
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

def to_device(x, device=def_device):
    if isinstance(x, torch.Tensor): return x.to(device)
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(to_device(o, device) for o in x)

def collate_device(b): return to_device(default_collate(b))

In [ ]:
#|export
class DataLoaders:
    def __init__(self,*dls): self.train,self.valid = dls[:2]
    @classmethod
    def from_dd(cls,dd,batch_size, as_tuple=True):
        return cls(*[DataLoader(ds, batch_size,num_workers=4, collate_fn=collate_dict(ds)) for ds in dd.values()])

In [9]:
dls = DataLoaders.from_dd(tds,bs)
dt = dls.train
xb,yb = next(iter(dt))
xb.shape,yb[:10]

(torch.Size([1024, 784]), tensor([5, 4, 9, 4, 3, 0, 6, 5, 7, 6]))

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()

    def one_batch(self):
        self.xb,self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()

    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs,self.losses,self.ns = [],[],[]
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [ ]:
m,nh =28*28,50
model = nn.Sequential(nn.Linear(m,nh),nn.ReLU(),nn.Linear(nh,10))

In [ ]:
#When the num_workers=0/4/8 the time it took largely changes 
learn = Learner(model,dls, F.cross_entropy, lr=0.2)
%time learn.fit(1)

## Basic Callback Learner

In [10]:
#|export
def identity(*args):
    #import pdb;pdb.set_trace()
    if not args:return
    x,*args = args
    return (x,)+tuple(args) if args else x

In [11]:
identity(3,'4',1)

(3, '4', 1)

In [12]:
#|export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [13]:
#| export
def run_cbs(cbs,method_nm):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method is not None:method()

In [86]:
#|export 
class callback(): order=0

In [87]:
class CompletionCallback(callback):
    def before_fit(self): self.count= 0
    def after_batch(self): self.count += 1
    def after_fit(self): print(f'Completed {self.count} batches')

In [88]:
cbs = [CompletionCallback()]
run_cbs(cbs, 'before_fit')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_fit')

Completed 1 batches


In [64]:
cb = cbs[0];cb

In [65]:
getattr(cb, 'after_fit', None)()

Completed 1 batches


In [19]:
#|export
class with_cbs:
    def __init__(self,nm): self.nm = nm
    def __call__(self, f):
        def _f(o,*args,**kwargs):
            try:
                o.callback(f"before_{self.nm}")
                f(o,*args,**kwargs)
                o.callback(f"After_{self.nm}")
            except globals()[f'Cancel{self.nm.title}Exception']:pass
        return _f

In [66]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs,opt_func=optim.SGD): 
        fc.store_attr()
        for cb in cbs: cb.learn = self

    def one_batch(self):
        self.preds = self.model(self.batch[0])
        self.loss = self.loss_func(self.preds, self.batch[1])
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()

    def one_epoch(self, train):
        self.model.training = train
        self.dl = self.dls.train if train else self.dls.valid
        try:
            self.callback('before_epoch')            
            for self.iter,self.batch in enumerate(self.dl): 
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException:pass
            self.callback('after_epoch')
        except CancelEpochException:pass
    
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try:
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException:pass

    def callback(self,method_nm): run_cbs(self.cbs,method_nm)

In [67]:
m,nh = 28*28,50
def get_model(): return nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [ ]:
class SingleCallback(callback):
    order =1
    def after_batch(self): raise CancelEpochException()

In [94]:
learn = Learner(get_model(),dls, F.cross_entropy, lr=0.2, cbs=[SingleCallback(),CompletionCallback()])
%time learn.fit(1)

Completed 2 batches
CPU times: user 400 ms, sys: 8.81 ms, total: 409 ms
Wall time: 417 ms


In [98]:
#| export
class DeviceCB(callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self): self.learn.model.to(self.device)
    def before_batch(self): self.learn.batch = to_device(self.learn.batch,device=self.device)

## Metric

In [80]:
#|export
class Metric:
    def __init__(self): self.reset()
    def reset(self): self.vals, self.ns = [],[]
    def add(self,inp, targ=None, n=1): # n is the number of items in the mini-batch.
        self.last = self.calc(inp,targ)
        self.vals.append(self.last)
        self.ns.append(n)
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    def calc(self, inps,targs): return inps 

In [81]:
#|export
class Accuracy(Metric):
    def calc(self,inps,targs): return (inps==targs).float().mean()

In [82]:
acc = Accuracy()
acc.add(tensor([0,1,2,0,1,2]),tensor([0,1,1,2,1,0]))
acc.add(tensor([1,1,2,0,1]),tensor([0,1,1,2,1]))
acc.value

tensor(0.45)

In [83]:
loss=Metric()
loss.add(0.6,n=32)
loss.add(0.9,n=2)
loss.value,round((0.6*32+0.9*2)/(32+2),2)

(tensor(0.62), 0.62)

In [96]:
#| export
from torcheval.metrics import MulticlassAccuracy,Mean

In [100]:
metric = MulticlassAccuracy()
metric.update(tensor([0,2,1,3]), tensor([0,1,2,3]))
metric.compute()

tensor(0.50)

In [99]:
metric.reset()
metric.compute()

tensor(nan)

In [110]:
#| export
def to_cpu(x):
    if isinstance(x,Mapping): return {k:to_cpu(v) for k,v in x.items()}
    if isinstance(x,list): return [to_cpu(o) for o in x]
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    return x.detach().cpu()

In [111]:
#| export
class MetricsCB(callback):
    def __init__(self, *ms, **metrics):
        for o in ms:metrics[type(o).__name__] = o
        self.metrics = metrics
        self.all_metrics = copy(metrics)
        self.all_metrics['loss'] = self.loss = Mean()
    def _log(self,d): print(d)
    def before_fit(self): self.learn.metrics = self
    def before_epoch(self): [o.reset() for o in self.all_metrics.values()]
    def after_epoch(self):
        log = {k:f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
        log['epoch'] = self.learn.epoch
        log['train'] = self.learn.model.training
        self._log(log)
    
    def after_batch(self):
        x,y = to_cpu(self.learn.batch)
        for m in self.metrics.values(): m.update(to_cpu(self.learn.preds),y)
        self.loss.update(to_cpu(self.learn.loss),weight=len(x))

In [112]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
learn = Learner(model,dls,F.cross_entropy, lr=0.2, cbs=[DeviceCB(),metrics])
learn.fit(1)

{'accuracy': '0.588', 'loss': '1.180', 'epoch': 0, 'train': True}
{'accuracy': '0.692', 'loss': '0.871', 'epoch': 0, 'train': False}


In [113]:
model = get_model()
metrics = MetricsCB(MulticlassAccuracy())
learn = Learner(model,dls,F.cross_entropy, lr=0.2, cbs=[DeviceCB(),metrics])
learn.fit(1)

{'MulticlassAccuracy': '0.615', 'loss': '1.153', 'epoch': 0, 'train': True}
{'MulticlassAccuracy': '0.694', 'loss': '0.856', 'epoch': 0, 'train': False}


### Flexible Learner

In [115]:
#| export
class Learner():
    def __init__(self,model,dls,loss_func,lr,cbs, opt_func= optim.SGD):
        fc.store_attr()
        for cb in cbs: cb.learn =self
     
    @contextmanager
    def callback_ctx(self,nm):
        try:
            self.callback(f'before_{nm}')
            yield
            self.callback(f'after_{nm}')
        except globals()[f'Cancel{nm.title}Exception']:pass
    
    def one_epoch(self,train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        with self.callback_ctx('epoch'):
            for self.iter,self.batch in enumerate(self.dl):
                 with self.callback_ctx('batch'):
                     self.predict()
                     self.get_loss()
                     if self.model.training:
                         self.backward()
                         self.step()
                         self.zero_grad()
                 
    def fit(self,n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(),self.lr)
        with self.callback_ctx('fit'):
            for self.epoch in self.epochs:
                 self.one_epoch(True)
                 self.one_epoch(False)
    
    def __getattribute__(self,name):
        if name in ('predict','get_loss','backward','step','zero_grad'): return partial(self.callback,name)
        raise AttributeError(name)
    
    def callback(self,method_nm): run_cbs(self.cbs,method_nm)

In [116]:
#| export
class TrainCb(callback):
    def predit(self): self.learn.preds = self.learn.model(self.learn.batch[0])
    def get_loss(self): self.learn.loss = self.learn.loss_func(self.learn.preds,self.learn.batch[1])
    def backward(self): self.learn.loss.backward()
    def step(self): self.learn.opt.step()
    def zero_grad(self): self.learn.opt.zero_grad()

In [ ]:
#|export
class Learner():
    def __init__(self,model,dls,loss_func,lr, cbs, opt_func=optim.SGD):
        @fc.store_attr()
        for cb in cbs: cb.learn= self
            
    @with_cbs('batch')
    def one_batch(self):
        self.predit()
        self.get_loss()
        if self.model.training:
            self.backward()
            self.step()
            self.zero_grad()
    
    def one_epoch(self,train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        self._one_epoch()
    
    @with_cb('epoch')
    def _one_epoch(self):
        for self.iter, self.batch in enumerate(self.dl): self.one_epoch()
    
    def fit(self, n_epochs):
        self.epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(),lr)
        self._fit()
        
    @with_cbs('fit')
    def _fit(self):
        for self.epoch in self.epochs:
            self.one_epoch(True)
            self.one_epoch(False)
    
    def __getattr__(self,name):
        if name in('predict','get_loss', 'backward','step','zero_grad'): return partial(self.callback,name)
        raise AttributeError(name)
    def callback(self,method_nm):
        for cb in sorted(self.cbs, key=attrgetter('order')): getattr(cb,method_nm,identity)()